In [1]:
import copy
import os
import numpy as np
from numpy.linalg import norm
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin"

# Path for .pkl files
# target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\"
target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\pilot_runs\\"
# Path for Quail source code
source_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\"

In [2]:
# Import quail modules
os.chdir(source_dir)
import argparse
import importlib
import sys

import defaultparams as default_deck
import errors
from general import ShapeType, SolverType, PhysicsType

import meshing.common as mesh_common
import meshing.gmsh as mesh_gmsh
import meshing.tools as mesh_tools

import numerics.helpers.helpers as helpers
import numerics.timestepping.tools as stepper_tools

import physics.zerodimensional.zerodimensional as zerod
import physics.euler.euler as euler
import physics.navierstokes.navierstokes as navierstokes
import physics.scalar.scalar as scalar
import physics.chemistry.chemistry as chemistry
import physics.multiphasevpT.multiphasevpT as multiphasevpT

import processing.readwritedatafiles as readwritedatafiles
import processing.post as post
import processing.plot as plot
import processing

import solver.DG as DG
import solver.ADERDG as ADERDG
import solver.tools as solver_tools

import time
import multiprocessing as mp
from multidomain import Domain, Observer

os.chdir(target_dir)

In [ ]:
%%capture
%run "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\quail" "conduit.py"
# Run box to run

In [ ]:
# Waves first arrive at left boundary of [-600,600] at ~ output20 (400ms)
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_21.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_35.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit2_35.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_35.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_52.pkl")
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_163.pkl")
#
#  for i in range(6):
#   plt.subplot(2,3,i+1)
#   plt.semilogy(solver.state_coeffs[:,:,i])

if solver.physics.NDIMS == 1:
	pltqtys = ["pDensityA", "pDensityWv", "pDensityM", "XMomentum",
		"Energy", "pDensityWt", "Pressure", "SoundSpeed", "Temperature",
		"XVelocity", "volFracM", "pDensityC"]
	equal_aspect_ratio = False
else:
	pltqtys = ["pDensityA", "XMomentum", "YMomentum", "Pressure", "Temperature"]
	equal_aspect_ratio = True

for pltqty in pltqtys:
	plot.plot_solution(solver.mesh, solver.physics, solver, pltqty,
				plot_numerical=True, plot_exact=False, plot_IC=False,
				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
				equidistant_pts=False, include_mesh=False, regular_2D=False,
				equal_AR=equal_aspect_ratio)

In [ ]:
idx = 163
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{idx}.pkl")
# plot.plot_solution(solver.mesh, solver.physics, solver, "Pressure",
# 				plot_numerical=True, plot_exact=False, plot_IC=False,
# 				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
# 				equidistant_pts=False, include_mesh=False, regular_2D=False,
# 				equal_AR=equal_aspect_ratio)
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm2_{idx}.pkl")
plot.plot_solution(solver.mesh, solver.physics, solver, "Pressure",
				plot_numerical=True, plot_exact=False, plot_IC=False,
				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
				equidistant_pts=False, include_mesh=False, regular_2D=False,
				equal_AR=equal_aspect_ratio)

In [ ]:
import ternary
%matplotlib tk

idx = 5
# solver_atm1 = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{idx}.pkl")
# solver_atm2 = readwritedatafiles.read_data_file(f"mixture_shocktube_atm2_{idx}.pkl")
# solver_conduit1 = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{idx}.pkl")
# solver_conduit2 = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit2_{idx}.pkl")
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{idx}.pkl")
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{idx}.pkl")
solver_conduit2 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit2_{idx}.pkl")

fontsize = 12

def prep_ternary():
  ## Boundary and Gridlines
  scale = 100.0
  figure, tax = ternary.figure(scale=scale)

  # Draw Boundary and Gridlines
  tax.boundary(linewidth=2.0)
  tax.gridlines(color="black", multiple=10)
  tax.gridlines(color="blue", multiple=5, linewidth=0.1)

  # Set Axis labels and Title
  # Set ticks
  tax.ticks(axis='lbr', multiple=10, linewidth=2)

  ternary.plt.show()
  plt.box(False)

  # Remove default Matplotlib Axes
  tax.clear_matplotlib_ticks()
  return tax

def get_wt_percents(solver, coordset="AWvM"):
  rho = np.sum(solver.state_coeffs[:,:,solver.physics.get_mass_slice()], axis=2)
  if coordset == "AWvM":
    y1 = solver.state_coeffs[:,:,0] / rho * 100.0
    y2 = solver.state_coeffs[:,:,1] / rho * 100.0
    y3 = solver.state_coeffs[:,:,2] / rho * 100.0
    return list(zip(y1.ravel(), y2.ravel(), y3.ravel()))
  elif coordset == "AWtMd":
    y1 = solver.state_coeffs[:,:,0] / rho * 100.0
    # Water transform
    arhoWt = solver.state_coeffs[:,:,solver.physics.get_state_index("pDensityWt")]
    arhoWd = arhoWt - solver.state_coeffs[:,:,1]
    y2 = arhoWt / rho * 100.0
    y3 = (solver.state_coeffs[:,:,2]-arhoWd) / rho * 100.0
  else:
    raise KeyError("Invalid coordset.")
  return list(zip(y1.ravel(), y2.ravel(), y3.ravel()))

''' Air, exsolved water, magma mixture'''
tax = prep_ternary()
tax.bottom_axis_label("wt% air", fontsize=fontsize)
tax.right_axis_label("wt% water, exsolved", fontsize=fontsize)
tax.left_axis_label("wt% frac melt + crystals + water, dissolved", fontsize=fontsize)

tax.right_corner_label("Air", fontsize=fontsize)
tax.top_corner_label("Exsolved water", fontsize=fontsize)
tax.left_corner_label("Magma", fontsize=fontsize)

tax.scatter(get_wt_percents(solver_atm1, "AWvM"), color='blue')
tax.scatter(get_wt_percents(solver_atm2, "AWvM"), color='red')
tax.scatter(get_wt_percents(solver_conduit1, "AWvM"), color='magenta')
tax.scatter(get_wt_percents(solver_conduit2, "AWvM"), color='cyan')
# tax.set_title("", fontsize=fontsize)

tax = prep_ternary()
tax.bottom_axis_label("wt% air", fontsize=fontsize)
tax.right_axis_label("wt% water, total", fontsize=fontsize)
tax.left_axis_label("wt% frac melt + crystals", fontsize=fontsize)

tax.right_corner_label("Air", fontsize=fontsize)
tax.top_corner_label("Total water", fontsize=fontsize)
tax.left_corner_label("Dry magma", fontsize=fontsize)

tax.scatter(get_wt_percents(solver_atm1, "AWtMd"), marker='d', color='blue', label="atm near")
tax.scatter(get_wt_percents(solver_atm2, "AWtMd"), marker='d', color='brown', label="atm far")
tax.scatter(get_wt_percents(solver_conduit1, "AWtMd"), marker='s', color='magenta', label="conduit upper")
tax.scatter(get_wt_percents(solver_conduit2, "AWtMd"), marker='s',color='cyan', label="conduit lower")

tax.legend()

In [ ]:
%matplotlib tk
idx = 120
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{idx}.pkl")
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{idx}.pkl")
solver_conduit2 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit2_{idx}.pkl")

from matplotlib.animation import FuncAnimation
fig = plt.figure()
# ax = plt.axes(xlim=(-2400, 0), ylim=(0, 100e6)) # p
ax = plt.axes(xlim=(-2400, 0), ylim=(-300, 500)) # u, c
# ax = plt.axes(xlim=(-2400, 0), ylim=(-1, 10e9)) # wv
# ax = plt.axes()
line, = ax.plot([], [], lw=2)

def get_nodes(solver):
  nodal_pts = solver.basis.get_nodes(solver.order)
  # Allocate [ne] x [nb, ndims]
  x = np.empty((solver.mesh.num_elems,) + nodal_pts.shape)
  for elem_ID in range(solver.mesh.num_elems):
    # Fill coordinates in physical space
    x[elem_ID] = mesh_tools.ref_to_phys(solver.mesh, elem_ID, nodal_pts)
  return x

def update(frame_index):
  frame_skip = 200
  stride = 10
  solver_conduit1 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_conduit1_{stride*frame_index+frame_skip}.pkl")
  x1 = get_nodes(solver_conduit1).ravel()
  p1 = solver_conduit1.physics.compute_additional_variable("Pressure", 
    solver_conduit1.state_coeffs, False).ravel()
  a1 = solver_conduit1.physics.compute_additional_variable("SoundSpeed", 
    solver_conduit1.state_coeffs, False).ravel()
  u1 = solver_conduit1.physics.compute_additional_variable("XVelocity", 
    solver_conduit1.state_coeffs, False).ravel()
  q1 = a1

  solver_conduit2 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_conduit2_{stride*frame_index+frame_skip}.pkl")
  x2 = get_nodes(solver_conduit2).ravel()
  p2 = solver_conduit2.physics.compute_additional_variable("Pressure", 
    solver_conduit2.state_coeffs, False).ravel()
  a2 = solver_conduit2.physics.compute_additional_variable("SoundSpeed", 
    solver_conduit2.state_coeffs, False).ravel()
  u2 = solver_conduit2.physics.compute_additional_variable("XVelocity", 
    solver_conduit2.state_coeffs, False).ravel()
  q2 = a2

  x = np.concatenate((x2, x1))
  p = np.concatenate((q2, q1))
  
  line.set_data(x, p)
  return line,

# plot.plot_solution(solver_conduit1.mesh, solver_conduit1.physics, solver_conduit1, "Pressure",
#   plot_numerical=True, plot_exact=False, plot_IC=False,
#   create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
#   equidistant_pts=False, include_mesh=False, regular_2D=False,
#   equal_AR=equal_aspect_ratio)
# plot.plot_solution(solver_conduit2.mesh, solver_conduit2.physics, solver_conduit2, "Pressure",
#   plot_numerical=True, plot_exact=False, plot_IC=False,
#   create_new_figure=False, ylabel=None, fmt='b.', legend_label="DG",
#   equidistant_pts=False, include_mesh=False, regular_2D=False,
#   equal_AR=equal_aspect_ratio)
# solver_conduit1.time

def init():
    line.set_data([], [])
    return line,

anim = FuncAnimation(fig, update, frames=100,
    init_func=init, blit=True)
  
# anim.save('basic_animation_test', fps=30)

plt.show()




In [18]:
%matplotlib tk
import processing.mdtools as mdt
solver_atm1_init = readwritedatafiles.read_data_file(f"v0_phi0600m_atm1_{0}.pkl")
solver_conduit1_init = readwritedatafiles.read_data_file(f"v0_phi0600m_conduit1_{0}.pkl")
mdt.viz(solver_atm1_init, "Pressure")
# solver_atm1_init.physics.compute_additional_variable("Pressure", solver_atm1_init.state_coeffs, False).max()
# solver_conduit1_init.physics.compute_additional_variable("Pressure", solver_conduit1_init.state_coeffs[-1:,:,:], False).min()

In [ ]:
equal_aspect_ratio = True
plot_idx = 1200
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{plot_idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{plot_idx}.pkl")
plot.plot_solution(solver_atm1.mesh, solver_atm1.physics, solver_atm1, "Velocity",
  plot_numerical=True, plot_exact=False, plot_IC=False,
  create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
  equidistant_pts=False, include_mesh=False, regular_2D=False,
  equal_AR=equal_aspect_ratio)
plot.plot_solution(solver_atm1.mesh, solver_atm1.physics, solver_atm2, "Velocity",
  plot_numerical=True, plot_exact=False, plot_IC=False,
  create_new_figure=False, ylabel=None, fmt='b.', legend_label="DG",
  equidistant_pts=False, include_mesh=False, regular_2D=False,
  equal_AR=equal_aspect_ratio)

In [ ]:
# cb = plt.colorbar()
# ax2 = cb.ax.twinx()
# ax2.set_ylim(clims)
plt.clf()
# sm = plt.cm.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]), cmap=plt.get_cmap())

In [ ]:
def downsample(solver, plot_qty:str="Pressure"):
	solver = copy.deepcopy(solver)
	# Downsample
	original_order = solver.order
	solver.order = 0
	equidistant_pts = True
	x = plot.get_sample_points(solver.mesh, solver, solver.physics, solver.basis,
				equidistant_pts)
	solver.order = original_order
	# Compute on downsampled points
	var_plot = plot.get_numerical_solution(solver.physics, solver.state_coeffs, x,
					solver.basis, plot_qty)
	return x, var_plot

def custom_plot_2D(x, var_plot, solver, levels=None):
	# plot.plot_2D_general(solver.physics, x, var_plot)

	if levels is None:
		''' Open temp figure to get default levels'''
		figtmp = plt.figure()
		# Run this for the sole purpose of getting default contour levels
		levels = plot.plot_2D_regular(solver.physics, np.copy(x), np.copy(var_plot))
		plt.close(figtmp)

	num_elems = x.shape[0]
	for elem_ID in range(num_elems):
		# Triangulate each element one-by-one
		tris, utri = plot.triangulate(solver.physics, x[elem_ID], var_plot[elem_ID])
		# Plot
		plt.tricontourf(tris, utri, levels=levels, extend="both")
		if np.mod(elem_ID,100) == 0:
			plt.pause(0.05)

	# Adjust plot
	plt.gca().set_aspect('equal', adjustable='box')
	plt.xlabel("$x$")
	plt.ylabel("$y$")
	return var_plot

def plot_mean(x, q, clims):
	''' Create very cheap plot '''
	cmap = plt.get_cmap()
	cnorm = matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]) 
	for i in range(x.shape[0]):
		pp = plt.Polygon([[x[i,0,0], x[i,0,1]],
			[x[i,1,0], x[i,1,1]],
			[x[i,2,0], x[i,2,1]]], facecolor=cmap(cnorm(q[i,:,0].mean())))
		plt.gca().add_patch(pp)
	plt.axis("auto")
	plt.axis("equal")

def plot_mean1D(x, q, clims, xscale=1.0, xshift=0.0):
	''' Create very cheap plot '''
	cmap = plt.get_cmap()
	cnorm = matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]) 
	for i in range(x.shape[0]):
		pp = plt.Rectangle((xshift, x[i,0,0]), width=xscale, height=x[i,1,0]-x[i,0,0],
			facecolor=cmap(cnorm(q[i,:,0].mean())))
		plt.gca().add_patch(pp)
	plt.axis("auto")
	plt.axis("equal")

plot_idx = 1200
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{plot_idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{plot_idx}.pkl")

x1, p1 = downsample(solver_atm1, "Pressure")
x2, p2 = downsample(solver_atm2, "Pressure")
clims = np.min([p1.min(), p2.min()]), np.max([p1.max(), p2.max()])
levels = np.linspace(clims[0], clims[1], 10)
# custom_plot_2D(x1, p1, solver_atm1, levels=levels)
# custom_plot_2D(x2, p2, solver_atm2, levels=levels)
plot_mean(x1, p1, clims)
plot_mean(x2, p2, clims)

sm = plt.cm.ScalarMappable(
	norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
	cmap=plt.get_cmap())
cb = plt.colorbar(sm)
cb.set_label("p (Pa)")


In [93]:
%matplotlib tk
from processing.mdtools import generate_anim

generate_anim(["v0_phi0500m_atm1", "v0_phi0500m_atm2"],
  ["v0_phi0500m_conduit1"],
  "output_p_0500",
  12,
  "Pressure",
  filter=lambda x, x0: np.log10(np.abs(x-x0)+1e-7),
  initial=1,
  stride=5,
  is_high_detail=False)

Animation constructed


In [208]:
plot_idx = 300
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{plot_idx}.pkl")
p = solver_conduit1.physics.compute_additional_variable("Pressure", solver_conduit1.state_coeffs, True)
T = solver_conduit1.physics.compute_additional_variable("Temperature", solver_conduit1.state_coeffs, True)
physics = solver_conduit1.physics

def mass_spec_entropy(T, p, Gas):
  c_p = Gas["c_p"]
  gamma = Gas["gamma"]
  return c_p * np.log(T / p**((gamma-1.0)/gamma))

s = (
  solver_conduit1.state_coeffs[:,:,0] * mass_spec_entropy(T, p, physics.Gas[0])[:,:,0]
  + solver_conduit1.state_coeffs[:,:,1] * mass_spec_entropy(T, p, physics.Gas[1])[:,:,0]
  + solver_conduit1.state_coeffs[:,:,2] * physics.Liquid["c_m"] * np.log(T)[:,:,0]
)

rho = solver_conduit1.state_coeffs[:,:,physics.get_mass_slice()].sum(axis=2)
S = s / rho

plt.clf()
plt.plot(S, '.-')

In [448]:
# Constructing pressure boundary condition using damped-Newton-RK
solver = solver_conduit1
bgroup = solver_atm1_init.mesh.boundary_groups["x2"]

U_init = solver.state_coeffs[solver.bface_helpers.elem_IDs[solver.mesh.boundary_groups["x2"].number], :, :]
# Modify initial to lower velocity
U = U_init.copy()
# Remove kinetic
U[:,:,4] -= 0.5 * U[:,:,3]**2 / U[:,:,0:3].sum(axis=2)
# Reduce momentum
U[:,:,3] /= 3.0
# Add back kinetic
U[:,:,4] += 0.5 * U[:,:,3]**2 / U[:,:,0:3].sum(axis=2)

# Define numerics
p_target = 1.6e6*2
damping_idx_scale = 100
additional_damping_factor = 1
N_steps_max = 100
# Permissive step count
N_steps_max = np.max([N_steps_max, 10*damping_idx_scale])
p_rel_tol = 1e-6

p = 1e-15
p_last = 1e15
stuck_counter = 0

def compute_eig_aco_negative(U, physics):
  ''' Computes eigenvector corresponding to u-c'''

  '''Compute rows of flux Jacobian for tracer states'''
  # Row of flux Jacobian for tracer states
  rho = U[:,:,physics.get_mass_slice()].sum(axis=2)
  u = U[:,:,physics.get_state_index("XMomentum")] / rho
  # Truncated row of flux Jacobian for tracer states
  #   b = [-u, -u, -u, 1, 0] / rho * q_i
  # where q_i is the partial density of the given tracer state
  N_states_hyperbolic = 5
  N_states_tracer = 2
  # Fill temporary construction vector with size [ne, nq, ns_hyp]
  b_sub = np.tile(np.zeros_like(u), (1,1,N_states_hyperbolic))
  b_sub[:,:,physics.get_mass_slice()] = -u/rho
  b_sub[:,:,physics.get_state_index("XMomentum")] = 1.0/rho
  # Fill temporary construction vector with size [ne, nq, ns_tracer]
  slice_like_tracers = (physics.get_state_index("pDensityWt"), 
    physics.get_state_index("pDensityC"))
  arho_tracers = U[:,:,slice_like_tracers]
  # Compute rows of flux Jacobian for tracer states
  b = np.einsum("ijk, ijl -> ijkl", arho_tracers, b_sub)

  ''' Compute u-c eigenvector of hyperbolic subsystem '''
  # Size [ne, nq, ns_hyp]
  #   x = y1 y2 y3 u-a H - au
  # Mass fractions
  y1 = U[:,:,0] / rho
  y2 = U[:,:,1] / rho
  y3 = U[:,:,2] / rho
  eigvec_hyp = np.zeros_like(b_sub)
  H = physics.compute_additional_variable("TotalEnthalpy", U, True)
  a = physics.compute_additional_variable("SoundSpeed", U, True)
  eigvec_hyp[:,:,0] = y1
  eigvec_hyp[:,:,1] = y2
  eigvec_hyp[:,:,2] = y3
  eigvec_hyp[:,:,3] = u - a
  eigvec_hyp[:,:,4] = H - a*u

  ''' Compute extension of the hyperbolic subsystem acoustic eigenvector'''
  # Compute (b^T * eigvec) / (eigval - u) -- 
  eigvec_ext = np.einsum("ijkl, ijl -> ijk", b, eigvec_hyp) / (-a)
  return np.concatenate((eigvec_hyp, eigvec_ext), axis=2)

p_fn = lambda U : solver.physics.compute_additional_variable("Pressure", U, True)
dpdU_fn = lambda U : solver.physics.compute_pressure_sgradient(U)
f = lambda U: compute_eig_aco_negative(U, solver.physics)

# Observation
p_hist = []
S_hist = []
y1_hist = []
y2_hist = []
y3_hist = []
yWt_hist = []
yC_hist = []
aux_hist = []
rho_hist = []
M_hist = []

for i in range(N_steps_max):
    ''' Set damped Newton step size '''
    # Discrete Gaussian damping
    damping = 1.0 - np.exp(-((i+1)/damping_idx_scale)**2)
    damping *= additional_damping_factor
    # Compute Newton step size
    newton_step_size = (p_target - p_fn(U)) / \
        np.einsum("ijk, ijk -> ij", dpdU_fn(U), f(U))
    
    # Check for stalling
    # if (p - p_last) / p_target < p_rel_tol and i > 2 * damping_idx_scale:
    #     # Increase damping (high damping for monotonic approach to target)
    #     # (Deterministic alternative to stochastic perturbation)
    #     damping *= (0.9)**(stuck_counter+1)
    #     stuck_counter += 1
    p_last = p

    # Butcher table for Cash-Karp RK quadrature
    B = np.array([[1/5, 0, 0, 0, 0],
         [3/40, 9/40, 0, 0, 0],
         [3/10, -9/10, 6/5, 0, 0],
         [-11/54, 5/2, -70/27, 35/27, 0],
         [1631/55296, 175/512, 575/13824, 44275/110592, 253/4096]])
    w = np.array([37/378, 0, 250/621, 125/594 , 0, 512/1771])
    # Compute damped step size for ODE integration
    damped_step_size = damping * newton_step_size
    # RK step 0
    num_stages = B.shape[0] + 1
    k = np.zeros(tuple(np.append([num_stages], list(U.shape))))
    k[0,:,:,:] = f(U)
    for j in range(B.shape[0]):
        k[j+1,:,:,:]= f(U + damped_step_size*
            np.einsum("m, mijk -> ijk", B[j,0:j+1], k[0:j+1,:]))
    U += damped_step_size * np.einsum("i, ijkl -> jkl", w, k)
    # U += damped_step_size * f(U)
    
    # Compute pressure
    p = p_fn(U)

    # q_hist(1+i,:) = q;
    p_hist.append(p)
    
    rho = U[:,:,0:3].sum(axis=2)
    y1 = U[:,:,0] / rho
    y2 = U[:,:,1] / rho
    y3 = U[:,:,2] / rho
    rhoA = U[:,:,0]/physics.compute_additional_variable("volFracA", U, True)
    rhoWv = U[:,:,1]/physics.compute_additional_variable("volFracWv", U, True)
    T = physics.compute_additional_variable("Temperature", U, True)
    S = y1 * solver.physics.Gas[0]["c_v"] * np.log(p / rhoA**solver.physics.Gas[0]["gamma"]) + \
        y2 * solver.physics.Gas[1]["c_v"] * np.log(p / rhoWv**solver.physics.Gas[1]["gamma"]) + \
        y3 * solver.physics.Liquid["c_m"] * np.log(T)
    
    a = solver.physics.compute_additional_variable("SoundSpeed", U, True)
    beta2 = solver.physics.compute_additional_variable("beta", U, True)
    velx = U[:,:,3] / rho
    # aux = U[:,:,3] / rho + 2 * a / \
      # (solver.physics.compute_additional_variable("Gamma", U, True) - 1)
    aux = U[:,:,3] / rho + a / \
      solver.physics.compute_additional_variable("beta", U, True)
    Mx = velx / a

    y1_hist.append(y1.squeeze())
    y2_hist.append(y2.squeeze())
    y3_hist.append(y3.squeeze())
    yWt_hist.append((U[:,:,5]/rho).squeeze())
    yC_hist.append((U[:,:,6]/rho).squeeze())
    rho_hist.append(rho.squeeze())
    S_hist.append(S.squeeze())
    aux_hist.append(aux.squeeze())
    M_hist.append(Mx.squeeze())
    # u2g1c_hist(1+i,1) = q(2) / q(1) ...
    #     + 2 / (gamma-1) * a(p_bind(q),q(1));

    
    #  Check termination tolerance
    if np.abs(p_target - p) / p_target < p_rel_tol:
        break


# solver.physics.compute_pressure_sgradient()


# solver.bface_helpers.x_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]
# quad_wts = solver_atm1_init.bface_helpers.quad_wts # [nq_bdry, 1]
# normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]

# # solver_atm1_init.
# Uq_synthetic = np.ones_like(solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number])[:,:,0:1] # [ne_bdry, nq_bdry, ns]
# # weighted_Uq = np.einsum('ijk, jm -> ijk', Uq_synthetic, quad_wts)

# bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
# normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number]
# # Compute boundary measure
# boundary_length = solver_atm1_init.bface_helpers.face_lengths_bgroups[bgroup.number].sum()
# # Compute average
# U_avg = np.einsum('ijm, ijk, jn -> k',
#   np.linalg.norm(normals, axis=2, keepdims=True),
#   Uq_synthetic,
#   solver_atm1_init.bface_helpers.quad_wts) / boundary_length

# # U_avg = U_avg/boundary_length
# U_avg

In [757]:
# np.einsum("ijk, ijk -> ij", dpdU_fn(U), f(U)), dpdU_fn(U), f(U)
# dpdU_fn(U).ravel() @ f(U).ravel()
# p_hist
# U[:,:,3] / U[:,:,0:3].sum(axis=2, keepdims=True)
# solver.physics.compute_additional_variable("SoundSpeed", U, True)

# dU = damped_step_size * np.einsum("i, ijkl -> jkl", w, k)
# f(U_init)
# f(U_init) - k[0,:,:]
# f(U_init + 1/5 * damped_step_size * k[0,:,:]) - k[1,:,:]
# f(U_init + 3/40 * damped_step_size * k[0,:,:] + 9/40 * damped_step_size * k[1,:,:]) - k[2,:,:]

# i
plt.figure(1)
plt.clf()
# plt.plot((np.array(yWt_hist)*np.array(aux_hist)).ravel())
plt.subplot(3,2,1)
plt.plot(np.array(S_hist).ravel())
plt.subplot(3,2,2)
plt.plot(np.array(y1_hist).ravel())
plt.subplot(3,2,3)
plt.plot(np.array(y2_hist).ravel())
plt.subplot(3,2,4)
plt.plot(np.array(yWt_hist).ravel())
plt.subplot(3,2,5)
plt.plot(np.array(yC_hist).ravel())
plt.subplot(3,2,6)
plt.plot(np.array(aux_hist).ravel())

for i in range(6):
  plt.subplot(3,2,i+1)
  plt.xlabel("Iter count")

plt.figure(2)
plt.clf()
plt.subplot(1,2,1)
plt.plot(np.array(p_hist).ravel())
plt.subplot(1,2,2)
plt.plot(np.array(M_hist).ravel())

p_fn(U_init)
solver.physics.compute_additional_variable("Psi1", U, True)

array([[[1.08217390826553]]])

In [774]:
# Testing the generalized Riemann invariant condition
# From
#   dq == r dt
# Implying
#   dPsi/dq . dq == dPsi/dq . r dt ?= 0
# Then if dPsi/dq . r == 0 then Psi is a generalized Riemann invariant
# (see Embid  & Baer '92)

def entropy(U, physics):
    rho = U[:,:,0:3].sum(axis=2)
    y1 = U[:,:,0] / rho
    y2 = U[:,:,1] / rho
    y3 = U[:,:,2] / rho
    rhoA = U[:,:,0]/physics.compute_additional_variable("volFracA", U, True).squeeze(axis=2)
    rhoWv = U[:,:,1]/physics.compute_additional_variable("volFracWv", U, True).squeeze(axis=2)
    p = physics.compute_additional_variable("Pressure", U, True).squeeze(axis=2)
    T = physics.compute_additional_variable("Temperature", U, True).squeeze(axis=2)
    S = y1 * solver.physics.Gas[0]["c_v"] * np.log(p / rhoA**solver.physics.Gas[0]["gamma"]) + \
        y2 * solver.physics.Gas[1]["c_v"] * np.log(p / rhoWv**solver.physics.Gas[1]["gamma"]) + \
        y3 * solver.physics.Liquid["c_m"] * np.log(T) \
    # Mixing related entropy, convex function of y1, y2
    # -x * log(x/(x+y)) has Hessian eigenvals 0, (x^2 + y^2) / (x (x + y)^2)
    S_mix = -y1 * solver.physics.Gas[0]["R"] * np.log(y1 * solver.physics.Gas[0]["R"] / (y1 * solver.physics.Gas[0]["R"] + y2 * solver.physics.Gas[1]["R"])) \
        -y2 * solver.physics.Gas[1]["R"] * np.log(y2 * solver.physics.Gas[1]["R"] / (y1 * solver.physics.Gas[0]["R"] + y2 * solver.physics.Gas[1]["R"]))
    return S

# Centered difference for gradient
def evaluate_orthogonality(U, diff_eps, physics, quant="acou", strat="cent2",
    is_normalized=True):
  if quant == "acou":
    # Set tentative Riemann invariant to u - a / beta
    Psi = lambda U : (U[:,:,3] / U[:,:,0:3].sum(axis=2)) + \
      solver.physics.compute_additional_variable("SoundSpeed", U, True).squeeze(axis=2) / \
      solver.physics.compute_additional_variable("beta", U, True).squeeze(axis=2)
  elif quant == "entropy":
    # Set tentative Riemann invariant to entropy
    Psi = lambda U : entropy(U, physics)
  elif quant == "y1":
    Psi = lambda U: U[:,:,0] / (U[:,:,0:3].sum(axis=2))
  elif quant == "y2":
    Psi = lambda U: U[:,:,1] / (U[:,:,0:3].sum(axis=2))
  elif quant == "y3":
    Psi = lambda U: U[:,:,2] / (U[:,:,0:3].sum(axis=2))
  elif quant == "pressure":
    Psi = lambda U: solver.physics.compute_additional_variable("Pressure", U, True).squeeze(axis=2)
    # Psi = lambda U: solver.physics.compute_additional_variable("SoundSpeed", U, True).squeeze(axis=2)
  else:
    raise Exception

  # Set finite difference step
  scaling_vec = diff_eps*U.squeeze()
  dU = np.expand_dims(np.diag(scaling_vec),axis=0)

  if strat == "cent2":
    scaling_vec[np.where(scaling_vec == 0)] = 1e-14
    # Centered difference
    gradPsi = (Psi((U+dU/2)) - Psi((U-dU/2))) / (scaling_vec)
  elif strat == "cent4":
    scaling_vec[np.where(scaling_vec == 0)] = 1e-14
    # HO Centered difference
    gradPsi = (
      -Psi(U+2*dU)
      +8*Psi(U+dU)
      -8*Psi(U-dU)
      +Psi(U-2*dU)) / (12*scaling_vec)
  else:
    raise Exception

  if is_normalized:
    x = compute_eig_aco_negative(U, physics).squeeze()
    return np.dot(gradPsi.squeeze(), x) / (np.linalg.norm(gradPsi.squeeze())
        * np.linalg.norm(x))
  else:
    return np.dot(gradPsi.squeeze(), 
      compute_eig_aco_negative(U, physics).squeeze())

# State vector to sus out
U_sus = U.copy()
# U_sus[:,:,2] = 0

diff_eps = np.logspace(-12,-1,200)
orthos = []
for i, e in enumerate(diff_eps):
  orthos.append(abs(evaluate_orthogonality(U_sus,e,solver.physics,quant="acou",strat="cent4")))
orthos = np.array(orthos)

plt.figure(3)
plt.clf()
line1, = plt.loglog(diff_eps, orthos, '.-', label="??")

orthos = []
for i, e in enumerate(diff_eps):
  orthos.append(abs(evaluate_orthogonality(U_sus,e,solver.physics,quant="entropy",strat="cent4")))
orthos = np.array(orthos)
line2, = plt.loglog(diff_eps, orthos, '.-', label="Mass-sum phasic entropy")

orthos = []
for i, e in enumerate(diff_eps):
  orthos.append(abs(evaluate_orthogonality(U_sus,e,solver.physics,quant="y1",strat="cent4")))
orthos = np.array(orthos)
line3, = plt.loglog(diff_eps, orthos, '.-', label="y_1")

orthos = []
for i, e in enumerate(diff_eps):
  orthos.append(abs(evaluate_orthogonality(U_sus,e,solver.physics,quant="y2",strat="cent4")))
orthos = np.array(orthos)
line4, = plt.loglog(diff_eps, orthos, '.-', label="y_2")

orthos = []
for i, e in enumerate(diff_eps):
  orthos.append(abs(evaluate_orthogonality(U_sus,e,solver.physics,quant="y3",strat="cent4")))
orthos = np.array(orthos)
line5, = plt.loglog(diff_eps, orthos, '.-', label="y_3")

orthos = []
for i, e in enumerate(diff_eps):
  orthos.append(abs(evaluate_orthogonality(U_sus,e,solver.physics,quant="pressure",strat="cent4")))
orthos = np.array(orthos)
line6, = plt.loglog(diff_eps, orthos, '.-', label="p")

# plt.ylim([1e-10, 1e5])
# orthos.min()
# solver.physics.compute_additional_variable("beta", U, True)

plt.xlabel("$h$")
plt.ylabel("$\\frac{\\nabla \\Psi \\cdot \\mathbf{r}} {|| \\nabla \\Psi || \\cdot  || \\mathbf{r} ||}$")
plt.legend(handles=[line1, line2, line3, line4, line5, line6])

In [783]:
# Solution sampling
sampling_range = range(375)
quant_keys = ["acou", "entropy", "y1", "y2", "y3", "pressure"]
d_quant_orthogs = {}
for quant in quant_keys:
  d_quant_orthogs[quant] = []
M_hist = []

for plot_idx in sampling_range:
  solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{plot_idx}.pkl")
  U_sampled = solver_conduit1.state_coeffs[solver.bface_helpers.elem_IDs[solver.mesh.boundary_groups["x2"].number], :, :]
  for quant in quant_keys:
    d_quant_orthogs[quant].append(abs(evaluate_orthogonality(U_sampled,1e-4,solver.physics,quant=quant,strat="cent4")))
  M_hist.append(solver_conduit1.physics.compute_additional_variable("XVelocity", U_sampled, True)
    / solver_conduit1.physics.compute_additional_variable("SoundSpeed", U_sampled, True)
  )


In [795]:

plt.figure(4)
plt.clf()
plt.subplot(2,1,1)
lines = []
for i, quant in enumerate(quant_keys):
  # Special naming for acou
  label = quant
  if quant == "acou":
    label = "$u + a/\\beta$"
  elif quant == "pressure":
    label = "p (negative control)"
  line, = plt.semilogy(np.array(d_quant_orthogs[quant]), '.-', label=label)
  lines.append(line)
plt.legend(handles=lines, bbox_to_anchor=(0.75, 0.5))
plt.title("Cosines at boundary for 2D1D coupled, sampled over saved files")
plt.ylabel("$\\frac{\\nabla \\Psi \\cdot \\mathbf{r}} {|| \\nabla \\Psi || \\cdot  || \\mathbf{r} ||}$")

plt.subplot(2,1,2)
plt.plot(np.array(M_hist).squeeze())
plt.xlabel("Save file index")
plt.ylabel("Mach")

Text(0, 0.5, 'Mach')

In [686]:
u1 = U_init[:,:,3] / U_init[:,:,0:3].sum()
a1 = solver.physics.compute_additional_variable("SoundSpeed", U_init, True)
beta1 = solver.physics.compute_additional_variable("beta", U_init, True)
Gamma1 = solver.physics.compute_additional_variable("Gamma", U_init, True)
u2 = U[:,:,3] / U[:,:,0:3].sum()
a2 = solver.physics.compute_additional_variable("SoundSpeed", U, True)
beta2 = solver.physics.compute_additional_variable("beta", U, True)
Gamma2 = solver.physics.compute_additional_variable("Gamma", U, True)
u1 + a1*2/(Gamma1-1), u2 + a2*2/(Gamma2-1)

(array([[[50726.11702483833]]]), array([[[52569.5517965487]]]))

In [239]:
def compute_eig(U, physics):
  '''Compute rows of flux Jacobian for tracer states'''
  # Row of flux Jacobian for tracer states
  rho = U[:,:,physics.get_mass_slice()].sum(axis=2)
  u = U[:,:,physics.get_state_index("XMomentum")] / rho
  # Truncated row of flux Jacobian for tracer states
  #   b = [-u, -u, -u, 1, 0] / rho * q_i
  # where q_i is the partial density of the given tracer state
  N_states_hyperbolic = 5
  N_states_tracer = 2
  # Fill temporary construction vector with size [ne, nq, ns_hyp]
  b_sub = np.tile(np.zeros_like(u), (1,1,N_states_hyperbolic))
  b_sub[:,:,physics.get_mass_slice()] = -u/rho
  b_sub[:,:,physics.get_state_index("XMomentum")] = 1.0/rho
  # Fill temporary construction vector with size [ne, nq, ns_tracer]
  slice_like_tracers = (physics.get_state_index("pDensityWt"), 
    physics.get_state_index("pDensityC"))
  arho_tracers = U[:,:,slice_like_tracers]
  # Compute rows of flux Jacobian for tracer states
  b = np.einsum("ijk, ijl -> ijkl", arho_tracers, b_sub)

  ''' Compute u-c eigenvector of hyperbolic subsystem '''
  # Size [ne, nq, ns_hyp]
  #   x = y1 y2 y3 u-a H - au
  # Mass fractions
  y1 = U[:,:,0] / rho
  y2 = U[:,:,1] / rho
  y3 = U[:,:,2] / rho
  eigvec_hyp = np.zeros_like(b_sub)
  H = physics.compute_additional_variable("TotalEnthalpy", U, True)
  a = physics.compute_additional_variable("SoundSpeed", U, True)
  eigvec_hyp[:,:,0] = y1
  eigvec_hyp[:,:,1] = y2
  eigvec_hyp[:,:,2] = y3
  eigvec_hyp[:,:,3] = u - a
  eigvec_hyp[:,:,4] = H - a*u

  ''' Compute extension of the hyperbolic subsystem acoustic eigenvector'''
  # Compute (b^T * eigvec) / (eigval - u) -- 
  eigvec_ext = np.einsum("ijkl, ijl -> ijk", b, eigvec_hyp) / (-a)

  return np.concatenate((eigvec_hyp, eigvec_ext), axis=2)

(1, 1, 7)

In [ ]:
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{plot_idx}.pkl")

xc1, pc1 = downsample(solver_conduit1, "Pressure")
plot_mean1D(xc1, pc1, clims, xscale=50, xshift=0.0)
plt.axis("auto")
plt.axis("equal")

In [91]:
solver_atm1_init.bface_helpers.quad_wts
# solver_atm1_init.bface_helpers.x_bgroups[bgroup.number]
bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
solver_atm1_init.bface_helpers.x_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]
quad_wts = solver_atm1_init.bface_helpers.quad_wts # [nq_bdry, 1]
normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]

# solver_atm1_init.
Uq_synthetic = np.ones_like(solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number])[:,:,0:1] # [ne_bdry, nq_bdry, ns]
# weighted_Uq = np.einsum('ijk, jm -> ijk', Uq_synthetic, quad_wts)

bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number]
# Compute boundary measure
boundary_length = solver_atm1_init.bface_helpers.face_lengths_bgroups[bgroup.number].sum()
# Compute average
U_avg = np.einsum('ijm, ijk, jn -> k',
  np.linalg.norm(normals, axis=2, keepdims=True),
  Uq_synthetic,
  solver_atm1_init.bface_helpers.quad_wts) / boundary_length

# U_avg = U_avg/boundary_length
U_avg

array([[[8.333333333274119],
        [8.333333333274119],
        [8.333333333274119]],

       [[8.333333333333336],
        [8.333333333333336],
        [8.333333333333336]],

       [[8.333333333392545],
        [8.333333333392545],
        [8.333333333392545]]])

In [ ]:
%matplotlib tk
initial = 0
stride = 10

fig = plt.figure()
# import celluloid
# import ffmpeg
# camera = celluloid.Camera(fig)
# plt.rcParams['animation.ffmpeg_path'] = 'C:\Users\User\Desktop'
plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin\\ffmpeg.exe"

# import ffmpeg
import matplotlib.animation as animation
FFwriter = animation.FFMpegWriter()
FFwriter.setup(plt.gcf(), "new_out_test_lol.mp4")

solver_atm1_init = readwritedatafiles.read_data_file(f"v0_phi0700m_atm1_{0}.pkl")
solver_atm2_init = readwritedatafiles.read_data_file(f"v0_phi0700m_atm2_{0}.pkl")
solver_conduit1_init = readwritedatafiles.read_data_file(f"v0_phi0700m_conduit1_{0}.pkl")

for i in range(120):
	plt.clf()
	read_index = initial+i*stride
	solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0700m_atm1_{read_index}.pkl")
	solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0700m_atm2_{read_index}.pkl")

	if False:
		''' Mass flux magnitude '''
		x1, p1 = downsample(solver_atm1, "XMomentum")
		x1, q1 = downsample(solver_atm1, "YMomentum")
		p1 = np.sqrt(p1**2+q1**2)
		x2, p2 = downsample(solver_atm2, "XMomentum")
		x2, q2 = downsample(solver_atm2, "YMomentum")
		p2 = np.sqrt(p2**2+q2**2)
	x1, p1 = downsample(solver_atm1, "Pressure")
	x2, p2 = downsample(solver_atm2, "Pressure")

	_, p1_init = downsample(solver_atm1_init, "Pressure")
	_, p2_init = downsample(solver_atm2_init, "Pressure")

	p1 -= p1_init
	p2 -= p2_init

	filter = lambda z : np.log10(np.abs(z)+1e-10)
	p1 = filter(p1)
	p2 = filter(p2)

	''' Conduit '''
	solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0700m_conduit1_{read_index}.pkl")
	xc1, pc1 = downsample(solver_conduit1, "Pressure")
	_, pc1_init = downsample(solver_conduit1_init, "Pressure")
	pc1 -= pc1_init
	pc1 = filter(pc1)

	# Auto clim
	clims = np.min([p1.min(), p2.min(), pc1.min()]), np.max([p1.max(), p2.max(), pc1.max()])

	clims = ([0,8])
	plot_mean(x1, p1, clims)
	plot_mean(x2, p2, clims)	
	
	plot_mean1D(xc1, pc1, clims, xscale=50, xshift=0.0)

	sm = plt.cm.ScalarMappable(
		norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
		cmap=plt.get_cmap())
	cb = plt.colorbar(sm)
	# cb.set_label("p (Pa)")
	
	plt.axis("auto")
	plt.axis("equal")
	plt.pause(0.010)
	# Grab frame with FFwriter
	FFwriter.grab_frame()

print("Done")
FFwriter.finish()

In [ ]:
%matplotlib tk

from matplotlib.animation import FuncAnimation
fig = plt.figure()
# ax = plt.axes(xlim=(-2400, 0), ylim=(0, 100e6)) # p
ax = plt.axes(xlim=(-2400, 0), ylim=(-300, 500)) # u, c
# ax = plt.axes(xlim=(-2400, 0), ylim=(-1, 10e9)) # wv
# ax = plt.axes()
line, = ax.plot([], [], lw=2)

def get_nodes(solver):
  nodal_pts = solver.basis.get_nodes(solver.order)
  # Allocate [ne] x [nb, ndims]
  x = np.empty((solver.mesh.num_elems,) + nodal_pts.shape)
  for elem_ID in range(solver.mesh.num_elems):
    # Fill coordinates in physical space
    x[elem_ID] = mesh_tools.ref_to_phys(solver.mesh, elem_ID, nodal_pts)
  return x

def update(frame_index):
  frame_skip = 200
  stride = 10
  solver_atm1 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_atm1_{stride*frame_index+frame_skip}.pkl")
  x1 = get_nodes(solver_atm1).ravel()
  p1 = solver_atm1.physics.compute_additional_variable("Pressure", 
    solver_atm1.state_coeffs, False).ravel()
  a1 = solver_atm1.physics.compute_additional_variable("SoundSpeed", 
    solver_atm1.state_coeffs, False).ravel()
  u1 = solver_atm1.physics.compute_additional_variable("XVelocity", 
    solver_atm1.state_coeffs, False).ravel()
  q1 = a1

  solver_atm2 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_atm2_{stride*frame_index+frame_skip}.pkl")
  x2 = get_nodes(solver_atm2).ravel()
  p2 = solver_atm2.physics.compute_additional_variable("Pressure", 
    solver_atm2.state_coeffs, False).ravel()
  a2 = solver_atm2.physics.compute_additional_variable("SoundSpeed", 
    solver_atm2.state_coeffs, False).ravel()
  u2 = solver_atm2.physics.compute_additional_variable("XVelocity", 
    solver_atm2.state_coeffs, False).ravel()
  q2 = a2

  x = np.concatenate((x2, x1))
  p = np.concatenate((q2, q1))
  plot_mean(x, p, clims)
  
  line.set_data(x, p)
  return line,

def init():
    line.set_data([], [])
    return line,

anim = FuncAnimation(fig, update, frames=100,
    init_func=init, blit=True)
  
# anim.save('basic_animation_test', fps=30)

plt.show()

In [ ]:
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_288.pkl")

if solver.physics.NDIMS == 1:
	pltqtys = ["pDensityA", "pDensityWv", "pDensityM", "XMomentum",
		"Energy", "pDensityWt", "Pressure", "SoundSpeed", "Temperature",
		"XVelocity", "volFracM", "pDensityC"]
	equal_aspect_ratio = False
else:
	pltqtys = ["pDensityA", "XMomentum", "YMomentum", "Pressure", "Temperature"]
	equal_aspect_ratio = True

for pltqty in pltqtys:
	plot.plot_solution(solver.mesh, solver.physics, solver, pltqty,
				plot_numerical=True, plot_exact=False, plot_IC=False,
				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
				equidistant_pts=False, include_mesh=False, regular_2D=False,
				equal_AR=equal_aspect_ratio)

In [ ]:
# Minimum of crystal partial density
np.min(solver.state_coeffs[:,:,6])

solver.state_coeffs[:,:,4]

In [ ]:
snapshotidx = 163
snapshotidx = 163

solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{snapshotidx}.pkl")
x_conduit = solver.mesh.node_coords
p_conduit = solver.physics.compute_additional_variable("Pressure", solver.state_coeffs, False)

solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{snapshotidx}.pkl")

p = solver.physics.compute_additional_variable("Pressure", solver.state_coeffs, False)
mesh = solver.mesh
# plt.plot(p.ravel())
# plt.scatter(mesh.node_coords[:,0], mesh.node_coords[:,:,1], p)
mesh.node_coords.shape
x = plot.get_sample_points(
			mesh, solver, solver.physics, solver.basis, True)
plt.scatter(x.mean(axis=1)[:,0], x.mean(axis=1)[:,1],
  c=p.mean(axis=1)[:,0])
# x.mean(axis=1).shape
# p.mean(axis=1)[:,0].shape
plt.axis("equal")

plt.figure()
# plt.plot(x.mean(axis=1)[:,1], p.mean(axis=1)[:,0], '.')
plt.plot(x.mean(axis=1)[:,1]+150.0, p[:,:,0], '.')
# p[:,:,0].shape

# Plot conduit
xnodes_conduit = np.zeros_like(p_conduit)
xnodes_conduit[:,0,:] = x_conduit[:-1]
xnodes_conduit[:,2,:] = x_conduit[1:]
xnodes_conduit[:,1,:] = 0.5*xnodes_conduit.sum(axis=1)
# plt.figure()
plt.plot(xnodes_conduit.ravel()-600, p_conduit.ravel() ,'.-')
plt.xlim([-100,100])
plt.ylim([5e4,300000])

In [ ]:
snapshotidx = 82
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{snapshotidx}.pkl")
num_saves = 82# 96
size_tuple = list(solver.state_coeffs[:,:,0:1].shape)
size_tuple.append(num_saves+1)
p_hist = np.zeros(tuple(size_tuple))
save_count = range(num_saves+1)
# x for 2D domain
x = plot.get_sample_points(
			solver.mesh, solver, solver.physics, solver.basis, True)

solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{snapshotidx}.pkl")
size_tuple = list(solver.state_coeffs[:,:,0:1].shape)
size_tuple.append(num_saves+1)
x_conduit = plot.get_sample_points(
			solver.mesh, solver, solver.physics, solver.basis, True)
p_conduit_hist = np.zeros(tuple(size_tuple))


key_quant = "Pressure"
for i in save_count:
  solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{i}.pkl")
  p_conduit_hist[:,:,:,i] = solver.physics.compute_additional_variable(key_quant, solver.state_coeffs, False)
  # p_conduit_hist[:,:,:,i] = solver.state_coeffs[:,:,2:3]
  solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{i}.pkl")
  p_hist[:,:,:,i] = solver.physics.compute_additional_variable(key_quant, solver.state_coeffs, False)
  # p_hist[:,:,:,i] = solver.state_coeffs[:,:,2:3]
  # Approximate energy
  # E_2D = np.sum(solver.state_coeffs[:,:,5])
  


# Problematic location
# plot.plot_solution(solver.mesh, solver.physics, solver, "Pressure",
# 				plot_numerical=True, plot_exact=False, plot_IC=False,
# 				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
# 				equidistant_pts=False, include_mesh=False, regular_2D=False,
# 				equal_AR=equal_aspect_ratio)
# plt.ylim([-150, -100])
# plt.xlim([0,50])
# plt.clim([-30000,60000*0])

# plt.figure()
# plt.plot(x.mean(axis=1)[:,1], p.mean(axis=1)[:,0], '.')
# plt.plot(x.mean(axis=1)[:,1]+150.0, p[:,:,0], '.')
# p[:,:,0].shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib tk
# p = solver.physics.compute_additional_variable("Pressure", solver.state_coeffs, False)
# p [258,:,:]
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

from matplotlib import cm

pseudo_x = x.mean(axis=1)[:,1]+150.0
ordering = np.argsort(pseudo_x)
X, Y = np.meshgrid(pseudo_x[ordering], save_count)
Z = p_hist[ordering,0,0,:].squeeze().T

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
# p_hist[:,:,-1]
# plt.xlabel("x")

ordering = np.argsort(pseudo_x)

plt.figure()
for i in (save_count):
  plt.plot(pseudo_x[ordering], p_hist[ordering,0,0,i], '.-')
  # plt.hold(True)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

x_conduit.mean(axis=1).shape
pseudo_x_conduit = x_conduit.mean(axis=1)[:,0]-200
p_conduit_hist.mean(axis=1)[:,0,:]

ordering = np.argsort(pseudo_x_conduit)
X, Y = np.meshgrid(pseudo_x_conduit[ordering], save_count)
Z = p_conduit_hist.mean(axis=1)[ordering,0,:].squeeze().T

# # Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)



In [ ]:
x_global = np.append(pseudo_x, pseudo_x_conduit)
p_global = np.append(p_hist.squeeze(), p_conduit_hist.mean(axis=1)[:,0,:], axis=0)
# p_hist.squeeze().shape
# p_conduit_hist.mean(axis=1)[:,0,:].shape

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# x_global.mean(axis=1).shape

ordering = np.argsort(x_global)
X, Y = np.meshgrid(x_global[ordering], save_count)
Z = p_global[ordering,:].squeeze().T

# # Plot the surface.
surf = ax.plot_surface(X, Y, np.log10(Z), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)




In [ ]:
assert(solver.physics.NDIMS == 2)
# np.argmin(p_hist[117,:,:])
p_hist[117,:,:,-1]
solver.state_coeffs[117,:,:]
# solver.physics.compute_additional_variable("Temperature", solver.state_coeffs, False)[258,:,:]